In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import dotenv
from sqlalchemy import create_engine, text

dotenv.load_dotenv()

True

In [3]:
# Start db
def create_conn():
    driver = os.getenv("DB_DRIVER")
    server = os.getenv("DB_SERVER")
    database = os.getenv("DB_NAME")
    trusted_connection = os.getenv("DB_TRUSTED_CONNECTION")

    return create_engine(
        f"mssql+pyodbc://{server}/{database}?trusted_connection={trusted_connection}&driver={driver}"
    )


engine = create_conn()
# Test connection
connection = engine.connect()
res = connection.execute(text("SELECT @@version;")).fetchone()
connection.close()
res[0]

'Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) \n\tOct  8 2022 05:58:25 \n\tCopyright (C) 2022 Microsoft Corporation\n\tDeveloper Edition (64-bit) on Windows 10 Home 10.0 <X64> (Build 19045: ) (Hypervisor)\n'

In [4]:
query_campaigns = """
SELECT
c.crm_campagne_id, c.crm_campagne_naam, c.crm_campagne_startdatum, c.crm_campagne_type_campagne, c.crm_campagne_soort_campagne, i.crm_contact_id
FROM campagne c INNER JOIN inschrijving AS i ON c.crm_campagne_id = i.crm_campagne_id WHERE 
c.crm_campagne_status = 'Actief'
"""

query_contacts = """
SELECT 
/*Contact*/
co.crm_contact_id, co.crm_contact_functietitel, crm_contact_voka_medewerker, 

/*Persoon*/
p.crm_persoon_mail_thema_duurzaamheid, p.crm_persoon_mail_thema_financieel_fiscaal, 
p.crm_persoon_mail_thema_innovatie, p.crm_persoon_mail_thema_internationaal_ondernemen, 
p.crm_persoon_mail_thema_mobiliteit, p.crm_persoon_mail_thema_omgeving, p.crm_persoon_mail_thema_sales_marketing_communicatie, p.crm_persoon_mail_thema_strategie_en_algemeen_management, 
p.crm_Persoon_Mail_thema_talent, p.crm_persoon_mail_thema_welzijn, p.crm_persoon_mail_type_bevraging, 
p.crm_persoon_mail_type_communities_en_projecten, p.crm_persoon_mail_type_netwerkevenementen, p.crm_persoon_mail_type_nieuwsbrieven, 
p.crm_persoon_mail_type_opleidingen, p.crm_persoon_mail_type_persberichten_belangrijke_meldingen, p.crm_persoon_marketingcommunicatie,

/*Account*/
a.crm_account_is_voka_entiteit, a.crm_account_ondernemingsaard, a.crm_account_ondernemingstype,
a.crm_account_primaire_activiteit, a.crm_account_id,

/*Financiele data*/
fd.crm_financieledata_toegevoegde_waarde, fd.crm_financieledata_fte, fd.crm_financieledata_aantal_maanden, fd.crm_financieledata_boekjaar,

/*Info & klachten*/
ik.aantal_klachten,

/*Activiteitscode*/
act.crm_activiteitscode_naam,

/*Afspraak betreft account*/
aba.crm_afspraak_betreft_account_thema, aba.crm_afspraak_betreft_account_subthema, aba.crm_afspraak_betreft_account_onderwerp,
aba.crm_afspraak_betreft_account_keyphrases, aba.crm_afspraak_id AS 'crm_afspraak_betreft_account_id',

/*Afspraak betreft contact*/
abc.crm_afspraak_id AS 'crm_afspraak_betreft_contact_id', abc.crm_afspraak_betreft_contactfiche_thema, abc.crm_afspraak_betreft_contactfiche_subthema, abc.crm_afspraak_betreft_contactfiche_onderwerp, abc.crm_afspraak_betreft_contactfiche_keyphrases

FROM contact AS co
INNER JOIN persoon AS p ON co.crm_persoon_id = p.crm_persoon_id
INNER JOIN account AS a ON a.crm_account_id = co.crm_contact_account
INNER JOIN financiele_data AS fd ON a.crm_account_id= fd.crm_account_id
INNER JOIN (
	SELECT COUNT(*) AS 'aantal_klachten', crm_account_id FROM info_en_klachten 
	GROUP BY crm_account_id
) AS ik ON ik.crm_account_id = a.crm_account_id
INNER JOIN account_activiteitscode AS aa ON a.crm_account_id= aa.crm_account_id
INNER JOIN activiteitscode AS act ON act.crm_activiteitscode_id = aa.crm_activiteitscode_id
INNER JOIN afspraak_betreft_account AS aba ON aba.crm_account_id = a.crm_account_id
INNER JOIN afspraak_betreft_contact AS abc ON abc.crm_contact_id = co.crm_contact_id
WHERE 
co.crm_contact_status = 'Actief' 
AND p.crm_persoon_reden_van_status = 'Actief'
AND a.crm_account_status = 'Actief'
AND act.crm_activiteitscode_status = 'Actief'
"""

In [5]:
df_campaigns = pd.read_sql(query_campaigns, engine)
df_campaigns

,crm_campagne_id,crm_campagne_naam,crm_campagne_startdatum,crm_campagne_type_campagne,crm_campagne_soort_campagne,crm_contact_id
0,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,6BE9AE3C-1B6D-E111-B43A-00505680000A
1,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,B61B99D1-34D0-EB11-8120-001DD8B72B61
2,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,A8A15926-6974-E111-B43A-00505680000A
3,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,48D9A655-5FD7-EC11-A7B5-000D3AACFF7A
4,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Infosessie,Online,BD6E0D2C-446B-E111-B43A-00505680000A
...,...,...,...,...,...,...
50910,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,70445417-E97C-EC11-8D21-6045BD8E0212
50911,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,2F33590A-3B6B-E111-B43A-00505680000A
50912,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,2DA76C1A-3B0F-EE11-8F6D-6045BD895CDC
50913,8A5277C6-1A59-EE11-BE6E-6045BD974EB2,OV-Laagdrempelige infosessie voor douane-gebru...,2023-10-25 15:30:00,Infosessie,Online,EFD66FB1-6073-E111-B43A-00505680000A


In [6]:
df_contacts = pd.read_sql(query_contacts, engine)
df_contacts

,crm_contact_id,crm_contact_functietitel,crm_contact_voka_medewerker,crm_persoon_mail_thema_duurzaamheid,crm_persoon_mail_thema_financieel_fiscaal,crm_persoon_mail_thema_innovatie,crm_persoon_mail_thema_internationaal_ondernemen,crm_persoon_mail_thema_mobiliteit,crm_persoon_mail_thema_omgeving,crm_persoon_mail_thema_sales_marketing_communicatie,...,crm_afspraak_betreft_account_thema,crm_afspraak_betreft_account_subthema,crm_afspraak_betreft_account_onderwerp,crm_afspraak_betreft_account_keyphrases,crm_afspraak_betreft_account_id,crm_afspraak_betreft_contact_id,crm_afspraak_betreft_contactfiche_thema,crm_afspraak_betreft_contactfiche_subthema,crm_afspraak_betreft_contactfiche_onderwerp,crm_afspraak_betreft_contactfiche_keyphrases
0,C8EFF01B-150B-E711-80E4-001DD8B72B62,Bedrijfsleider,0,Ja,Nee,Nee,Nee,Nee,Nee,Ja,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"[NAME] [NAME] [NAME] , sterke usp , privéconce...",D69D313F-56F1-E811-80FA-001DD8B72B62,D86CBDA2-1EE7-EC11-BB3C-00224880A534,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,"uitdaging minder [NAME] [NAME] , gedeeltelijke..."
1,C8EFF01B-150B-E711-80E4-001DD8B72B62,Bedrijfsleider,0,Ja,Nee,Nee,Nee,Nee,Nee,Ja,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"[NAME] [NAME] [NAME] , sterke usp , privéconce...",D69D313F-56F1-E811-80FA-001DD8B72B62,D86CBDA2-1EE7-EC11-BB3C-00224880A534,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,"uitdaging minder [NAME] [NAME] , gedeeltelijke..."
2,C8EFF01B-150B-E711-80E4-001DD8B72B62,Bedrijfsleider,0,Ja,Nee,Nee,Nee,Nee,Nee,Ja,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"[NAME] [NAME] [NAME] , sterke usp , privéconce...",D69D313F-56F1-E811-80FA-001DD8B72B62,D86CBDA2-1EE7-EC11-BB3C-00224880A534,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,"uitdaging minder [NAME] [NAME] , gedeeltelijke..."
3,C8EFF01B-150B-E711-80E4-001DD8B72B62,Bedrijfsleider,0,Ja,Nee,Nee,Nee,Nee,Nee,Ja,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"[NAME] [NAME] [NAME] , sterke usp , privéconce...",D69D313F-56F1-E811-80FA-001DD8B72B62,D86CBDA2-1EE7-EC11-BB3C-00224880A534,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,"uitdaging minder [NAME] [NAME] , gedeeltelijke..."
4,C8EFF01B-150B-E711-80E4-001DD8B72B62,Bedrijfsleider,0,Ja,Nee,Nee,Nee,Nee,Nee,Ja,...,Lidmaatschap,Retentie (Lidmaatschap),OV-ledenbezoek-2018,"[NAME] [NAME] [NAME] , sterke usp , privéconce...",D69D313F-56F1-E811-80FA-001DD8B72B62,D86CBDA2-1EE7-EC11-BB3C-00224880A534,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022,"uitdaging minder [NAME] [NAME] , gedeeltelijke..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32120,722A59C1-4D75-EC11-8943-000D3ABB8FEA,CEO,0,Nee,Nee,Nee,Nee,Nee,Nee,Nee,...,Innovatie,Andere (Innovatie),OV-Innovatie-Gastbedrijf I4.0Lab2021,"berry bpi formipac , algemene toelichting 3d -...",E10127CD-7AF3-EB11-8125-001DD8B72B62,0628C741-FDDA-ED11-A7C7-6045BD8F0F71,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2023,"belangrijkste items , technisch textiel , [NAM..."
32121,89D72CD0-23E3-EB11-8121-001DD8B72B61,Talent Development & Learning Director,0,Nee,Nee,Nee,Nee,Nee,Nee,Nee,...,Innovatie,Open innovatie,OV-Voorbereidend bezoek businessclub innovatie,"[NAME] [NAME] [NAME] , businessclub innovatie ...",71ACF1FE-DC88-E811-80F2-001DD8B72B62,7E174BD7-D079-EC11-8D21-000D3ABB8787,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022 C-level,"[NAME] [NAME] [NAME] cathy verstraete , andere..."
32122,89D72CD0-23E3-EB11-8121-001DD8B72B61,Talent Development & Learning Director,0,Nee,Nee,Nee,Nee,Nee,Nee,Nee,...,Innovatie,Open innovatie,OV-Voorbereidend bezoek businessclub innovatie,"[NAME] [NAME] [NAME] , businessclub innovatie ...",71ACF1FE-DC88-E811-80F2-001DD8B72B62,7E174BD7-D079-EC11-8D21-000D3ABB8787,Lidmaatschap,Retentie (Lidmaatschap),ov-ledenbezoek-2022 C-level,"[NAME] [NAME] [NAME] cathy verstraete , andere..."
32123,0742FE06-A86B-E111-B43A-005056800

In [7]:
result_df = pd.merge(df_campaigns, df_contacts.reset_index(), on="crm_contact_id")
# Delete all rows from result_df

all_dfs = []

count = 0
df_o = pd.merge(
    df_campaigns, df_contacts.reset_index(), left_on="crm_contact_id", right_on="crm_contact_id"
)
df_o["attended"] = 1

def remove_redundant_boekjaar(df):
    # Filter out the rows where crm_boekjaar != YEAR(crm_campagne_startdatum)
    df["crm_campagne_startdatum"] = pd.to_datetime(df["crm_campagne_startdatum"])
    df["crm_campagne_startdatum_year"] = df["crm_campagne_startdatum"].dt.year

    df = df[df["crm_campagne_startdatum_year"] == df["crm_financieledata_boekjaar"]]
    df = df.drop(columns=["crm_campagne_startdatum_year"])
    return df

df_o = remove_redundant_boekjaar(df_o)

for idx, row in df_contacts.iterrows():
    for idx2, row2 in df_campaigns.iterrows():
        print(f"{count}/{len(df_o)} ({count/len(df_o)*100:.2f}%)", end="\r")
        if row["crm_contact_id"] == row2["crm_contact_id"]:
            continue
            
        # Get all columns from row2 and add them to row
        merged_row = {**row.to_dict(), **row2.to_dict()}
        merged_row = pd.DataFrame(merged_row, index=[0])
        merged_row["attended"] = 0
        all_dfs.append(merged_row)
        count += 1
        if count == len(df_o):
            break
    if count == len(df_o):
        break
result_df = pd.concat(all_dfs)
# Concat with df_o
result_df = pd.concat([df_o, result_df])
# Sample 
result_df = result_df.sample(frac=1).reset_index(drop=True)
result_df

In [ ]:
# Get the activiteitcode on 1 row
df_activiteitcode = result_df[["crm_account_id", "crm_activiteitscode_naam"]]
df_activiteitcode = df_activiteitcode.pivot_table(
    index="crm_account_id", columns="crm_activiteitscode_naam", aggfunc="size"
).reset_index()
# All NaN values are replaced by 0
# All values bigger than 0 are replaced by 1
df_activiteitcode = df_activiteitcode.fillna(0)
# All numbered values are replaced by 
df_activiteitcode = df_activiteitcode.map(lambda x: 1 if not isinstance(x, str) and 0 < x  else x)
df_activiteitcode

In [ ]:
df = result_df.drop(columns=["crm_activiteitscode_naam"])
df = pd.merge(df, df_activiteitcode, left_on="crm_account_id", right_on="crm_account_id")
df

In [ ]:
def parse_keyphrases(
    df,
    anchor_name,
    column_name,
):
    # Create a dataframe with the keyphrases
    df_keyphrases = df[[anchor_name, column_name]].copy()

    # Get all distinct campagne_types
    campagne_types = df["crm_campagne_type_campagne"].unique()
    for campagne_type in campagne_types:
        # Create a new column for each campagne_type
        df_keyphrases.loc[:, anchor_name + "_" + campagne_type] = df_keyphrases.apply(
            lambda x: 1
            if isinstance(x[column_name], str) and (campagne_type.lower() in x[column_name].lower())
            else 0,
            axis=1,
        )

    # Drop the original column
    df_keyphrases = df_keyphrases.drop(columns=[column_name])
    # Join back into the main dataframe
    df = df.drop(columns=[anchor_name])
    df = pd.merge(df, df_keyphrases, left_index=True, right_index=True)
    return df


df = parse_keyphrases(
    df, "crm_afspraak_betreft_account_id", "crm_afspraak_betreft_account_keyphrases"
)
df = parse_keyphrases(
    df,
    "crm_afspraak_betreft_contact_id",
    "crm_afspraak_betreft_contactfiche_keyphrases",
)
df

In [ ]:
def parse_dataframe(df):
    # Filter out the rows where crm_boekjaar != YEAR(crm_campagne_startdatum)
    df["crm_campagne_startdatum"] = pd.to_datetime(df["crm_campagne_startdatum"])
    df["crm_campagne_startdatum_year"] = df["crm_campagne_startdatum"].dt.year

    df = df[df["crm_campagne_startdatum_year"] == df["crm_financieledata_boekjaar"]]
    df = df.drop(columns=["crm_campagne_startdatum_year"])
    # Get the activiteitcode on 1 row

    df_activiteitcode = df[["crm_account_id", "crm_activiteitscode_naam"]]
    df_activiteitcode = df_activiteitcode.pivot_table(
        index="crm_account_id", columns="crm_activiteitscode_naam", aggfunc="size"
    ).reset_index()
    # All NaN values are replaced by 0
    # All values bigger than 0 are replaced by 1
    df_activiteitcode = df_activiteitcode.fillna(0)
    # All numbered values are replaced by
    df_activiteitcode = df_activiteitcode.map(
        lambda x: 1 if not isinstance(x, str) and 0 < x else x
    )
    # Bring the activiteitcode back into the main dataframe


    df = df.drop(columns=["crm_activiteitscode_naam"])
    df = pd.merge(
        df, df_activiteitcode, left_on="crm_account_id", right_on="crm_account_id"
    )
    df = parse_keyphrases(
        df, "crm_afspraak_betreft_account_id", "crm_afspraak_betreft_account_keyphrases"
    )
    df = parse_keyphrases(
        df,
        "crm_afspraak_betreft_contact_id",
        "crm_afspraak_betreft_contactfiche_keyphrases",
    )
    return df

df_sample = parse_dataframe(df_sample)
df_o = parse_dataframe(df_o)